# Quantum Programming Lab3

In [1]:
from qiskit import *
from qiskit_aer import AerSimulator
import numpy as np
from qiskit.quantum_info import Operator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

## Part 1: Quantum circuit for Shor's algorithm

Construct a compiled version of quantum circuit for Shor's algorithm.

Shor's algorithm consists of the following steps; choose a co-prime 
$a$, where $a \in [2,N-1]$ and the greatest common divisor of  $a$ and $N$ is 1, find the order of $a$ modulo $N$, the smallest integer $r$
such that $a^r \mod N=1$, and then obtain the factor of $N$ by computing the greatst common divisor of $a^{r/2} \pm 1$ and $N$. In this procedure, the second step, finding the order of $a$ modulo $N$, is the only quantum part, quantum order-finding.

In Ch.3.9 Shor's Algorithm https://github.com/Qiskit/textbook/blob/main/notebooks/ch-algorithms/shor.ipynb,
we built a quantum circuit to find the order for $a=7$ and $N=15$. However, as we are very well aware by now, such a large depth circuit is not practical to run on near-term quantum systems due to the presence of noise. Here in part 1 of this lab, we construct a practical quantum circuit for the same example, which could generate a meaningful solution when executed on today's quantum computers.

In general, the quantum order-finding circuit to factorize the number $N$ requires $m=\log_2(N)$ qubits in the computational (auxiliary) register and 
$2m(=t)$ qubit in the period (counting) registers .i.e. total $3m$ qubits, at minimum. Therefore, $12$ qubits were used in the quantum circuit to factorize the number 15 in Shor's Algorithm. In addition, the cotrolled unitary operator for the modular function, $f(x)=a^{x} \mod N$ was applied in a cascading manner as shown in the figure below to produce the highly entangled state $\sum_{x=0}^{2^m-1}\ket{x}\ket{a^x \mod N}$, which increseas the circuit depth substantially. However the size of the circuit can be reduced based on several observations.

![title](Figure/L7_Circ_gen.svg)

### 1.Remove redundancy

### StepA:  Run the following cell to create the gate $U$ for the function $7\mod15$.
The unitary operator $U$ is defined as $U\ket{x}\equiv \ket{7x(\mod 15)}$

In [11]:
## Create 7mod15 gate
N = 15
m = int(np.ceil(np.log2(N)))

U_qc = QuantumCircuit(m)
U_qc.x(range(m))
U_qc.swap(1, 2)
U_qc.swap(2, 3)
U_qc.swap(0, 3)

U = U_qc.to_gate()
U.name ='{}Mod{}'.format(7, N)

Confirm if the unitary operator works properly by creating a quantum circuit with $m$ qubits. Prepare the input state representing any integer between 0 and 15 (exclusive) such as $\ket{1}(=\ket{0001})$, $\ket{5}(=\ket{0101})$, $\ket{13}(=\ket{0101})$ etc, and apply $U$ gate on it. Check if the circuit produces the expected outcomes for several inputs. The outcome state for the input $\ket{1}$ should be $\ket{7}(=\ket{0111})$ and 
 $\ket{1}$ for the input $\ket{13}$, for example.

In [3]:
## Your code here

### StepB:  Create a quantum circuit with $m$ qubits implementing $U$ gate $4=2^2$ times and run it using Operator(Which was imported in the first code block) to obtain the matrix resprentation of the gates in the circuit. Verify $U^{2^2}=I$


As shown in the above figure, modular exponentiation is realized by implementing the controlled unitary operator on each qubit 
$2^n$ times in series when $n$ goes from $0$ to $7$ for our example. However, we will find out that whole sets of operations are redundant when 
$n>1$ for $7\mod15$ case, hence the redundant operation can be removed from the circuit.

In [4]:
## Your code here

### StepC: Run the cells below to see the reduced circuit, shor_QPE, and execute it on the qasm_simulator to check if it reproduce the estimated phases in the Qiskit textbook https://github.com/Qiskit/textbook/blob/main/notebooks/ch-algorithms/quantum-phase-estimation.ipynb

In [10]:
def cU_multi(k):
    circ = QuantumCircuit(m)
    for _ in range(2**k):
        circ.append(U, range(m))
    
    U_multi = circ.to_gate()
    U_multi.name = '7Mod15_[2^{}]'.format(k)
    
    cU_multi = U_multi.control()
    return cU_multi

In [4]:
def qft(n):
    """Creates an n-qubit QFT circuit"""
    circuit = QuantumCircuit(n)
    def swap_registers(circuit, n):
        for qubit in range(n//2):
            circuit.swap(qubit, n-qubit-1)
        return circuit
    def qft_rotations(circuit, n):
        """Performs qft on the first n qubits in circuit (without swaps)"""
        if n == 0:
            return circuit
        n -= 1
        circuit.h(n)
        for qubit in range(n):
            circuit.cp(np.pi/2**(n-qubit), qubit, n)
        qft_rotations(circuit, n)
    
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

In [ ]:
# QPE circuit for Shor
t = 3 
shor_QPE = QuantumCircuit(t+m, t)
shor_QPE.h(range(t))

shor_QPE.x(t)
for idx in range(t-1):
    shor_QPE.append(cU_multi(idx), [idx]+ list(range(t,t+m)))

qft_dag = qft(t).inverse()
qft_dag.name = 'QFT+'

shor_QPE.append(qft_dag, range(t))
shor_QPE.measure(range(t), range(t))

shor_QPE.draw()

In [ ]:
backend = AerSimulator()

transpiled_circuit = transpile(shor_QPE, backend)
job = backend.run(transpiled_circuit, shots=1000, memory=True)
count_QPE= job.result().get_counts()

key_new = [str(int(key,2)/2**3) for key in count_QPE.keys()]
count_new_QPE = dict(zip(key_new, count_QPE.values()))
plot_histogram(count_new_QPE)

## Part 2: Noisy simulation of the quantum order-finding circuits.

Goal

Perform the noise simulaton of all two quantum order-finding circuits: shor_Orig, shor_QPE. 
Compare their results.
After this experiment, hopefully you will get better understand why we still need quantum error correction to factor large integer.
A famous paper that I recommend you to read:
https://arxiv.org/abs/1905.09749

In [ ]:
t = 2*m

shor_Orig = QuantumCircuit(t+m, t)
shor_Orig.h(range(t))

shor_Orig.x(t)
for idx in range(t):
    shor_Orig.append(cU_multi(idx), [idx]+ list(range(t,t+m)))

qft_dag = qft(t).inverse()
qft_dag.name = 'QFT+'

shor_Orig.append(qft_dag, range(t))
shor_Orig.measure(range(t), range(t))
    
shor_Orig.draw()

In [ ]:
backend = AerSimulator()

transpiled_circuit = transpile(shor_Orig, backend)
job = backend.run(transpiled_circuit, shots=1000, memory=True)
count_Orig= job.result().get_counts()

key_new = [str(int(key,2)/2**t) for key in count_Orig.keys()]
count_Orig = dict(zip(key_new, count_Orig.values()))
plot_histogram(count_Orig, title='textbook circuit simulation result No noise')

#### Your task: Perform the noise simulations of all two circuits, shor_Orig, shor_QPE, with the noise model that we provide you here and run your simulation for different noise parameters p =[0.1,0.2,0.3,0.4,0.5] plot their noise simulation results together with ones without noise for comparison.

In [38]:
# Import from Qiskit Aer noise module
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)


def construct_bitphaseflip_noise_model(p):
    # We set three different noise parameter the same as p
    p_reset = p
    p_meas = p
    p_gate1 = p

    #Phase flip noise when there is gate of measurement
    error_reset = pauli_error([('Z', p_reset), ('I', 1 - p_reset)])
    error_meas = pauli_error([('Z',p_meas), ('I', 1 - p_meas)])
    error_gate1 = pauli_error([('Z',p_gate1), ('I', 1 - p_gate1)])
    error_gate2 = error_gate1.tensor(error_gate1)

    #Bitflip noise when there is gate of measurement
    error_reset_bit = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
    error_meas_bit = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1_bit = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
    error_gate2_bit = error_gate1_bit.tensor(error_gate1)

    # Add above errors to the same noise model object
    noise_bitphase_flip = NoiseModel()
    noise_bitphase_flip.add_all_qubit_quantum_error(error_reset_bit, "reset")
    noise_bitphase_flip.add_all_qubit_quantum_error(error_meas_bit, "measure")
    noise_bitphase_flip.add_all_qubit_quantum_error(error_gate1_bit, ["x","h","t","tdg"])
    noise_bitphase_flip.add_all_qubit_quantum_error(error_gate2_bit, ["cx,ccx"])
    

    return noise_bitphase_flip

Setup your noisemodel with noise parameter

In [43]:
p=0
noisemodel=construct_bitphaseflip_noise_model(p)

In [ ]:
backend = AerSimulator()

transpiled_circuit = transpile(shor_Orig, backend)
job = backend.run(transpiled_circuit,noise_model=noisemodel, shots=1000, memory=True)
count_Orig= job.result().get_counts()

key_new = [str(int(key,2)/2**t) for key in count_Orig.keys()]
count_Orig = dict(zip(key_new, count_Orig.values()))
plot_histogram(count_Orig, title='textbook circuit simulation with noise')

## Additional task: Do the lab two more times, but these times for N=21 and N=63. Describe any major differences in behavior between these cases.

First, we add a modification of function cU_multi to _cU_multi, by setting both U and m as parameters:

In [53]:
def _cU_multi(U,m,k):
    circ = QuantumCircuit(m)
    for _ in range(2**k):
        circ.append(U, range(m))
    
    U_multi = circ.to_gate()
    U_multi.name = '7Mod15_[2^{}]'.format(k)
    
    cU_multi = U_multi.control()
    return cU_multi

Here I give you an example of the Shor15 function. This is the exact format we expect you to implement for N=21 and N=63.
You might use any base $a$ as you like for modular exponential. 

In [54]:
def Shor15():
    ## Code for constructing shor_QPE for factoring 15 starts here
    a = 7
    N = 15
    m = int(np.ceil(np.log2(N)))
    
    U_qc = QuantumCircuit(m)
    U_qc.x(range(m))
    U_qc.swap(1, 2)
    U_qc.swap(2, 3)
    U_qc.swap(0, 3)
    
    U = U_qc.to_gate()
    U.name ='{}Mod{}'.format(7, N)

    # QPE circuit for Shor15
    t = 3 
    shor_QPE = QuantumCircuit(t+m, t)
    shor_QPE.h(range(t))
    
    shor_QPE.x(t)
    for idx in range(t-1):
        shor_QPE.append(_cU_multi(U,m,idx), [idx]+ list(range(t,t+m)))
    
    qft_dag = qft(t).inverse()
    qft_dag.name = 'QFT+'
    
    shor_QPE.append(qft_dag, range(t))
    shor_QPE.measure(range(t), range(t))
    ## Code for constructing shor_QPE for factoring 15 ends here    

    
    backend = AerSimulator()
    transpiled_circuit = transpile(shor_QPE, backend)
    job = backend.run(transpiled_circuit, shots=1000, memory=True)
    count_QPE= job.result().get_counts()
    return a, shor_QPE, count_QPE

Please implement the following function of factoring 21. 
You should submit this function in .py file.

In [55]:
def Shor21():
    ##Your circuit for creating quantum circuit shor_QPT of factoring 21 starts here
    a = 2 # By default, you can use a=2
    shor_QPE=None




    
    ##Your circuit for creating quantum circuit shor_QPT of factoring 21 ends here    
    backend = AerSimulator()
    transpiled_circuit = transpile(shor_QPE, backend)
    job = backend.run(transpiled_circuit, shots=1000, memory=True)
    count_QPE= job.result().get_counts()
    return a, shor_QPE, count_QPE

Please implement the following function of factoring 21. 
You should submit this function in .py file.

In [56]:
def Shor63():
    ##Your circuit for creating quantum circuit shor_QPT of factoring 63 starts here
    shor_QPE=None
    a = 2 # By default, you can use a=2    




    
    ##Your circuit for creating quantum circuit shor_QPT of factoring 63 ends here    
    backend = AerSimulator()
    transpiled_circuit = transpile(shor_QPE, backend)
    job = backend.run(transpiled_circuit, shots=1000, memory=True)
    count_QPE= job.result().get_counts()
    return a, shor_QPE, count_QPE